In [9]:
import pandas as pd
import gene_conversion_tools as gct

## Load HPRD Raw Data
#### Source: http://www.hprd.org/download
#### The file requires registration with the database. Download the file: HPRD_Release9_041310.tar.gz
Downloaded: August 12, 2016  
Last Updated: June 29, 2010  
The following files are manipulated after unzipping the .tar.gz file

In [10]:
wd = '/cellar/users/jkhuang/Data/Projects/Network_Analysis/Data/'
HPRD_Raw = pd.read_csv(wd+'Network_Data_Raw/HPRD_Release9_062910/BINARY_PROTEIN_PROTEIN_INTERACTIONS.txt',sep='\t',header=-1)

In [11]:
# Assign column names from README file from archive
HPRD_Raw.columns = ['Interactor 1 Gene Symbol', 'Interactor 1 HPRD ID', 'Interactor 1 RefSeq ID',
                    'Interactor 2 Gene Symbol', 'Interactor 2 HPRD ID', 'Interactor 2 RefSeq ID',
                    'Experiment Type', 'PubMed ID']

In [12]:
# Use RefSeq ID and Gene Symbol 
HPRD_Raw_Genes_Symbol = set(HPRD_Raw['Interactor 1 Gene Symbol']).union(set(HPRD_Raw['Interactor 2 Gene Symbol']))
HPRD_Raw_Genes_RefSeq = set(HPRD_Raw['Interactor 1 RefSeq ID']).union(set(HPRD_Raw['Interactor 2 RefSeq ID']))
HPRD_Raw_Genes = HPRD_Raw_Genes_Symbol.union(HPRD_Raw_Genes_RefSeq)

In [13]:
# Get edge list of network
query_edgelist = HPRD_Raw[['Interactor 1 Gene Symbol', 'Interactor 1 RefSeq ID','Interactor 2 Gene Symbol','Interactor 2 RefSeq ID']].values.tolist()
print len(query_edgelist), "HPRD Edges"

39240 HPRD Edges


## Convert Genes

In [14]:
query_string, valid_genes, invalid_genes = gct.query_constructor(HPRD_Raw_Genes)

19290 Valid Query Genes
0 Invalid Query Genes


In [15]:
# Set scopes (gene naming systems to search)
scopes = "refseq, symbol, alias, retired"

# Set fields (systems from which to return gene names from)
fields = "symbol, entrezgene"

In [16]:
# Query MyGene.Info
match_list = gct.query_batch(query_string, scopes=scopes, fields=fields)
print len(match_list), 'Matched query results'

Batch query complete: 80.3 seconds
20693 Matched query results


In [17]:
match_table_trim, query_to_symbol, query_to_entrez = gct.construct_query_map_table(match_list, valid_genes)

Queries without full matching results found: 446

896 Queries with mutliple matches found

Query mapping table/dictionary construction complete: 139.24 seconds


## Construct Converted Network

In [18]:
# Separate each edge in to two lists of node identifiers to be passed into conversion function
query_edgelist_split = [[[edge[0],edge[1]],[edge[2],edge[3]]] for edge in query_edgelist]

In [19]:
# This is a custom gene conversion function written due to the parsing required for gene interactor labels
# Returns best matched symbol and/or entrez id from each HPRD interactor string (if applicable)
def convert_HPRD_list(names, field):
    # Keep only mappings defined for field of interest
    if field=='symbol':
        # Return match table values that have matched symbol
        conversion = match_table_trim.ix[names][~(match_table_trim.ix[names]['Symbol'].isnull())]
        if conversion.shape[0]==0:
            return None
        else:
            # Return conversion with max score or None if no conversion
            max_score = conversion['Score'].max()
            return conversion[conversion['Score']==max_score].ix[0]['Symbol']
    elif field=='entrez':
        # Return match table values that have matched symbol
        conversion = match_table_trim.ix[names][~(match_table_trim.ix[names]['EntrezID'].isnull())]
        if conversion.shape[0]==0:
            return None
        else:
            # Return conversion with max score or None if no conversion
            max_score = conversion['Score'].max()
            return conversion[conversion['Score']==max_score].ix[0]['EntrezID']

In [20]:
HPRD_edgelist_symbol = [sorted([convert_HPRD_list(edge[0],'symbol'),convert_HPRD_list(edge[1],'symbol')]) for edge in query_edgelist_split]
HPRD_edgelist_entrez = [sorted([convert_HPRD_list(edge[0],'entrez'),convert_HPRD_list(edge[1],'entrez')]) for edge in query_edgelist_split]

In [21]:
HPRD_edgelist_symbol_filt = gct.filter_converted_edgelist(HPRD_edgelist_symbol)
gct.write_edgelist(HPRD_edgelist_symbol_filt, wd+'Network_SIFs_Symbol/HPRD_Symbol.sif')

39240 input edges
2160 self-edges removed
59 edges with un-mapped genes removed
24 duplicate edges removed
Edge list filtered: 0.07 seconds
36997 Edges remaining
Edge list saved: 0.08 seconds


In [22]:
HPRD_edgelist_entrez_filt = gct.filter_converted_edgelist(HPRD_edgelist_entrez)
gct.write_edgelist(HPRD_edgelist_entrez_filt, wd+'Network_SIFs_Entrez/HPRD_Entrez.sif')

39240 input edges
2160 self-edges removed
59 edges with un-mapped genes removed
24 duplicate edges removed
Edge list filtered: 0.06 seconds
36997 Edges remaining
Edge list saved: 0.05 seconds
